<a href="https://colab.research.google.com/github/Ahmed8501/Masters/blob/main/notebooks/masters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import openai
import json
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity


In [2]:
!git clone https://github.com/Ahmed8501/Masters.git


Cloning into 'Masters'...
remote: Enumerating objects: 74, done.
remote: Counting objects: 100% (74/74), done.
remote: Compressing objects: 100% (71/71), done.
remote: Total 74 (delta 26), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (74/74), 1.18 MiB | 2.42 MiB/s, done.
Resolving deltas: 100% (26/26), done.


In [3]:
%cd Masters/notebooks


/content/Masters/notebooks


In [5]:
from google.colab import drive
drive.mount('/content/drive')

# Navigate to the upload location
%cd /content/drive/MyDrive/spider_data

# Unzip it
# !unzip -q database.zip


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/spider_data


## Checking spider data

In [7]:
# Check the number of entries in train_spider.json
with open('/content/Masters/data/spider/train_spider.json') as f:
    train_data = json.load(f)

print(len(train_data))

7000


In [9]:
# Check the number of entries in dev.json
with open('/content/Masters/data/spider/dev.json') as f:
    train_data = json.load(f)

print(len(train_data))

1034


In [8]:
# Check the number of schemas in tables.json

with open('/content/Masters/data/spider/tables.json') as f:
    train_data = json.load(f)

print(len(train_data))

166


In [10]:
sample = train_data[0]
print(sample['question'])
print(sample['sql'])
print(sample['db_id'])


How many singers do we have?
{'from': {'table_units': [['table_unit', 1]], 'conds': []}, 'select': [False, [[3, [0, [0, 0, False], None]]]], 'where': [], 'groupBy': [], 'having': [], 'orderBy': [], 'limit': None, 'intersect': None, 'union': None, 'except': None}
concert_singer


In [12]:
#Checking tables in a specific database
db_path = '/content/drive/MyDrive/spider_data/database/concert_singer'
import sqlite3

conn = sqlite3.connect(f'{db_path}/concert_singer.sqlite')
cursor = conn.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

print("Tables in concert_singer:", tables)


Tables in concert_singer: [('stadium',), ('singer',), ('concert',), ('singer_in_concert',)]


In [13]:
# This lists:

# column names
# data types
# if it's a primary key

cursor.execute("PRAGMA table_info(singer);")
print(cursor.fetchall())


[(0, 'Singer_ID', 'INT', 0, None, 1), (1, 'Name', 'TEXT', 0, None, 0), (2, 'Country', 'TEXT', 0, None, 0), (3, 'Song_Name', 'TEXT', 0, None, 0), (4, 'Song_release_year', 'TEXT', 0, None, 0), (5, 'Age', 'INT', 0, None, 0), (6, 'Is_male', 'bool', 0, None, 0)]


## **Schema Linking Annotations**